<a href="https://colab.research.google.com/github/AbdElRahmanFarhan/box_semantic_segmentation/blob/main/hyperparameter_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycocotools
!pip install coco-eval
!pip install wandb

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 135.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.4 MB/s eta 0:00:00
  Created wheel for coco-eval: filename=coco_eval-0.0.4-py3-none-any.whl size=5016 sha256=dd40f88aa8a57ec42ecccfdb303be6ee4ed71f51e105464a7bde5017325db3f3
  S

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import wandb
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abdofarhan75 (abdelrahman-farhan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
import zipfile

zip_file_path = '/content/drive/MyDrive/OSCD.zip'
dataset_folder = '/content/drive/MyDrive/OSCD/'

if len(os.listdir(dataset_folder)) == 0:
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      zip_ref.extractall(dataset_folder)

  print(f"Unzipped to: {dataset_folder}")

In [5]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_folder = os.path.join(dataset_folder, 'coco_carton/oneclass_carton/images/train2017')
val_folder = os.path.join(dataset_folder, 'coco_carton/oneclass_carton/images/val2017')
train_annotation = os.path.join(dataset_folder, 'coco_carton/oneclass_carton/annotations/instances_train2017.json')
val_annotation = os.path.join(dataset_folder, 'coco_carton/oneclass_carton/annotations/instances_val2017.json')

In [6]:
import torch
from torchvision.datasets import CocoDetection
import torchvision.transforms as T
from PIL import Image
from torchvision.tv_tensors import Mask
from torchvision.transforms.functional import to_tensor
from skimage.draw import polygon as sk_polygon


class OSCDDataset(CocoDetection):
    def __init__(self, img_folder, ann_file):
        super().__init__(img_folder, ann_file, transforms=None)

    def __len__(self) -> int:
       return super().__len__()

    def __getitem__(self, idx):
        img, anns = super().__getitem__(idx)

        if anns:
          labels = []
          areas = []
          iscrowd = []
          masks = []
          boxes = []
          ids = []
          for ann in anns:
              x, y, w, h = list(map(int, ann['bbox']))
              boxes.append([x, y, x + w, y + h])
              labels.append(ann['category_id'])
              areas.append(ann['area'])
              iscrowd.append(ann['iscrowd'])
              mask = self.get_mask(ann['segmentation'], img.size[1], img.size[0])
              masks.append(mask)
              ids.append(ann['id'])

          labels = torch.tensor(labels, dtype=torch.int64)
          areas = torch.tensor(areas, dtype=torch.float16)
          iscrowd = torch.tensor(iscrowd, dtype=torch.uint8)
          boxes = torch.tensor(boxes, dtype=torch.int64)
          masks = torch.stack(masks, dim=0)
          ids = torch.tensor(ids, dtype=torch.int64)
          img_id = torch.tensor(ann['image_id'], dtype=torch.int64)

          target = {
              "boxes": boxes,
              "labels": labels,
              "image_id": img_id,
              "ids": ids,
              "area": areas, # TODO: is it area or areas
              "iscrowd": iscrowd,
              "masks": Mask(masks),
          }
        else:
          target = {}
        img = to_tensor(img) # TODO: return an empty image
        return img, target

    def get_mask(self, segmentation, height, width):
        mask = torch.zeros((height, width), dtype=torch.bool)
        poly_x = segmentation[0][::2]
        poly_y = segmentation[0][1::2]
        rr, cc = sk_polygon(poly_y, poly_x, shape=(height, width))
        mask[rr, cc] = 1
        return mask


In [7]:
from torchvision.transforms import v2 as T
train_dataset = OSCDDataset(train_folder, train_annotation)
val_dataset = OSCDDataset(val_folder, val_annotation)
train_dataset_small = torch.utils.data.Subset(train_dataset, list(range(100)))
val_dataset_small = torch.utils.data.Subset(val_dataset, list(range(10)))

loading annotations into memory...
Done (t=4.31s)
creating index...
index created!
loading annotations into memory...
Done (t=1.05s)
creating index...
index created!


In [8]:
def collate_fn(batch):
    images, targets = [], []
    for (image, target) in batch:
      if not target:
        continue
      else:
        images.append(image)
        targets.append(target)
    return images, targets

In [9]:
# import matplotlib.pyplot as plt
# from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
# train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn, pin_memory=True, num_workers=2)
# val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn, pin_memory=True)
# images, targets = next(iter(val_loader))
# images = [image for image in images]
# targets = [{k: v for k, v in t.items()} for t in targets]
# img = images[0]
# target = targets[0]
# # plt_img = draw_bounding_boxes(img, target_boxes, colors="red")
# masks = (target["masks"] > 0.9).squeeze(1)
# plt_img = draw_segmentation_masks(img, masks, alpha=0.5, colors="blue")
# plt.figure(figsize=(12, 12))
# plt.imshow(plt_img.cpu().permute(1, 2, 0))
# plt.axis('off')
# plt.show()

In [10]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)

in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, 256, 2)

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:00<00:00, 233MB/s]


In [11]:
# import torchvision
# from torchvision.models.detection import MaskRCNN
# from torchvision.models.detection.backbone_utils import BackboneWithFPN
# from torchvision.models.resnet import resnet18
# from torchvision.models.detection.backbone_utils import BackboneWithFPN

# resnet = resnet18(pretrained=True)
# backbone = torch.nn.Sequential(
#     resnet.conv1,
#     resnet.bn1,
#     resnet.relu,
#     resnet.maxpool,
#     resnet.layer1,
#     resnet.layer2,
#     resnet.layer3,
#     resnet.layer4
# )

# return_layers = {
#     '4': '0',  # layer1
#     '5': '1',  # layer2
#     '6': '2',  # layer3
#     '7': '3',  # layer4
# }

# in_channels_list = [64, 128, 256, 512]
# out_channels = 256

# fpn_backbone = BackboneWithFPN(
#     backbone,
#     return_layers=return_layers,
#     in_channels_list=in_channels_list,
#     out_channels=out_channels
# )

# model = MaskRCNN(backbone=fpn_backbone, num_classes=2)

In [12]:
# model

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [14]:
import math
from tqdm.auto import tqdm
from torch.amp import autocast, GradScaler

def run_epoch(model, dataloader, optimizer, device, scaler, is_training):
    model.train()
    progress_bar = tqdm(total=len(dataloader), desc="Train" if is_training else "Valid")  # Initialize a progress bar
    batch_counter = 0
    epoch_total_loss = 0.
    epoch_losses = {
      'loss_classifier': 0,
      'loss_box_reg': 0.,
      'loss_mask': 0.,
      'loss_objectness': 0.,
      'loss_rpn_box_reg': 0.}
    for batch_id, (images, targets) in enumerate(dataloader):

        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        if len(targets) == 0:
          continue

        batch_counter += 1
        with autocast(device_type=device.type, dtype=torch.bfloat16):
            if is_training:
                losses = model(images, targets)
            else:
                with torch.no_grad():
                    losses = model(images, targets)

            total_loss = sum([loss for loss in losses.values()])

        if is_training:
            optimizer.zero_grad()
            if scaler:
                scaler.scale(total_loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                total_loss.backward()
                optimizer.step()

        epoch_losses = {k: v.item() + epoch_losses[k] for k, v in losses.items()}
        epoch_total_loss += total_loss.item()
        progress_bar_dict = dict(avg_loss=epoch_total_loss/(batch_counter+1))
        progress_bar.set_postfix(progress_bar_dict)
        progress_bar.update()
        if is_training:
          assert not math.isnan(total_loss.item()) and math.isfinite(total_loss.item()), "Loss is NaN or infinite. Stopping training."
    progress_bar.close()
    epoch_losses = {k: v/(batch_counter + 1) for k, v in epoch_losses.items()}
    return epoch_losses

In [15]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'valid/loss',
    'goal': 'minimize'
    }

sweep_config['metric'] = metric
parameters_dict = {
    'epochs': {
        'values': [10]
        },
    'lr': {
        'values': [1e-5]
        },
    'weight_decay': {
          'values': [1e-2]
        },
    'bs': {
          'values': [2]
        },
    'save_model_every': {
          'values': [10]
        },
    'scheduler': {
          'values': ['step']
        },
    'step_size': {
          'values': [40]
        },
    'gamma': {
          'values': [0.1]
        },
    'optimizer_type': {
          'values': ['adamw']
        },
    'rpn_batch_size_per_image': {
          'values': [128]
        },
    'rpn_positive_fraction': {
          'values': [0.7]
        },
    'rpn_nms_thresh': {
          'values': [0.7]
        },
    'rpn_fg_iou_thresh': {
          'values': [0.7]
        },
    'rpn_bg_iou_thresh': {
          'values': [0.3]
        },
    }

sweep_config['parameters'] = parameters_dict

In [16]:
sweep_id = wandb.sweep(sweep_config, project="box_segmentation")

Create sweep with ID: m4bo00qb
Sweep URL: https://wandb.ai/abdelrahman-farhan/box_segmentation/sweeps/m4bo00qb


In [17]:
from torch.optim import AdamW
import datetime
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def train(config=None):
  with wandb.init(config=config):
    config = wandb.config
    lr = config.lr
    weight_decay = config.weight_decay
    epochs = config.epochs
    bs = config.bs
    save_every = config.save_model_every

    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT",
                                                                rpn_batch_size_per_image=config.rpn_batch_size_per_image,
                                                                rpn_positive_fraction=config.rpn_positive_fraction,
                                                                rpn_nms_thresh=config.rpn_nms_thresh,
                                                                rpn_fg_iou_thresh=config.rpn_fg_iou_thresh,
                                                                rpn_bg_iou_thresh=config.rpn_bg_iou_thresh,

    )
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)

    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, 256, 2)

    model.to(device)
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer_type = config.optimizer_type

    if optimizer_type == 'sgd':
      optimizer = torch.optim.SGD(params, lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_type == 'adamw':
      optimizer = torch.optim.AdamW(params, lr=lr, weight_decay=weight_decay)

    scheduler = config.scheduler
    scheduler_step = config.step_size
    gamma = config.gamma
    if scheduler == 'step':
      lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=gamma)
    elif scheduler == 'linear':
      lr_scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=epochs)
    elif scheduler == 'cyclic':
      lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr, total_steps=epochs)


    train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True, collate_fn=collate_fn, pin_memory=True, num_workers=12)
    val_loader = DataLoader(val_dataset, batch_size=bs, shuffle=True, collate_fn=collate_fn, pin_memory=True, num_workers=12)

    scaler = GradScaler()
    for epoch in tqdm(range(epochs), desc="Epochs"):

        train_losses = run_epoch(model, train_loader, optimizer, device, scaler, is_training=True)

        with torch.no_grad():
            valid_losses = run_epoch(model, val_loader, None, device, scaler, is_training=False)
        lr_scheduler.step()

        train_losses = {f'train/{k}': v for k, v in train_losses.items()}
        wandb.log(train_losses)
        train_loss = sum(train_losses.values())
        wandb.log({'train/loss': train_loss})

        valid_losses = {f'valid/{k}': v for k, v in valid_losses.items()}
        wandb.log(valid_losses)
        valid_loss = sum(valid_losses.values())
        wandb.log({'valid/loss': valid_loss})

        wandb.log({'lr': lr_scheduler.get_last_lr()[0]})

        if (epoch+1) % save_every == 0:
          model_path = os.path.join(dataset_folder, 'model', f'model_{wandb.run.sweep_id}_{(epoch+1)}.pth')
          torch.save(model.state_dict(), model_path)

In [ ]:
wandb.agent(sweep_id, train, count=1)

wandb: Agent Starting Run: ij8lxdgs with config:
wandb: 	bs: 2
wandb: 	epochs: 10
wandb: 	gamma: 0.1
wandb: 	lr: 1e-05
wandb: 	optimizer_type: adamw
wandb: 	rpn_batch_size_per_image: 128
wandb: 	rpn_bg_iou_thresh: 0.3
wandb: 	rpn_fg_iou_thresh: 0.7
wandb: 	rpn_nms_thresh: 0.7
wandb: 	rpn_positive_fraction: 0.7
wandb: 	save_model_every: 10
wandb: 	scheduler: step
wandb: 	step_size: 40
wandb: 	weight_decay: 0.01


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Train:   0%|          | 0/3701 [00:00<?, ?it/s]

Valid:   0%|          | 0/500 [00:00<?, ?it/s]

Train:   0%|          | 0/3701 [00:00<?, ?it/s]

Valid:   0%|          | 0/500 [00:00<?, ?it/s]

Train:   0%|          | 0/3701 [00:00<?, ?it/s]

Valid:   0%|          | 0/500 [00:00<?, ?it/s]

Train:   0%|          | 0/3701 [00:00<?, ?it/s]